In [1]:
from IPython.display import Image
import ee, folium
ee.Initialize()

In [25]:
collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2').select('NO2_column_number_density').filterDate('2019-06-01', '2019-06-06')
band_viz = {
  'min': 0,
  'max': 0.0002,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
roi = ee.Geometry.Rectangle([65, 5, 97, 37])

In [32]:
mapid = collection.mean().clip(roi).getMapId(band_viz)

map = folium.Map(location=[24.11,75.27],zoom_start=4, height=500,width=700)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NO2',
  ).add_to(map)

map.add_child(folium.LayerControl())

map.save('plot_data.html')
map

In [27]:
def LatLonImg(img):
    img = img.addBands(ee.Image.pixelLonLat())
 
    img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=roi,\
                                        maxPixels=1e13,\
                                        scale=10);
 
    data = np.array((ee.Array(img.get("result")).getInfo()))
    lats = np.array((ee.Array(img.get("latitude")).getInfo()))
    lons = np.array((ee.Array(img.get("longitude")).getInfo()))
    return lats, lons, data

In [29]:
# import numpy as np
# lats, lons, data = LatLonImg(collection.mean())